<a href="https://colab.research.google.com/github/Champjiwrd/CE63-21-ObjectDetection/blob/master/mobilenetv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [2]:
%tensorflow_version 1.x  #Select module of the tensorflow
!pip show tensorflow

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  #Select module of the tensorflow`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.7
Requires: keras-applications, wrapt, astor, wheel, google-pasta, opt-einsum, termcolor, grpcio, tensorboard, six, keras-preprocessing, protobuf, absl-py, gast, tensorflow-estimator, numpy
Required-by: stable-baselines, magenta, fancyimpute


In [3]:
!pip install tf_slim

     |████████████████████████████████| 358kB 5.7MB/s 


In [4]:
%cd /root/
!git clone https://github.com/tensorflow/models.git # Import required models from Github

/root
Cloning into 'models'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 53889 (delta 11), reused 22 (delta 1), pack-reused 53857
Receiving objects: 100% (53889/53889), 569.76 MiB | 31.37 MiB/s, done.
Resolving deltas: 100% (37008/37008), done.


In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-03-19 06:54:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.159.124.229, 52.55.225.227, 52.5.36.149, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.159.124.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  37.4MB/s    in 0.4s    

2021-03-19 06:54:15 (37.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [8]:
#the logs that are created while training 
LOG_DIR = "/root/models/trained"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://ec14c05d452f.ngrok.io


In [9]:
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'
!python object_detection/builders/model_builder_test.py

/root/models/research


# GG ln

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
!ln -s /content/gdrive/MyDrive/CEProject63-21ObjectDetection/colab_work/ /ce2
!ln -s /content/gdrive/MyDrive/CEProject63-21ObjectDetection/dataset/ /dataset2

In [16]:
%cd /ce2

/content/gdrive/.shortcut-targets-by-id/1hQoqbcLdIKP1y1vE2waDf28rLEbA3yqR/CEProject63-21ObjectDetection/colab_work


# yolo to csv

In [80]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image

In [77]:
os.chdir(r'/content/gdrive/MyDrive/CEProject63-21ObjectDetection/dataset/t1')
myFiles = glob.glob('*.txt')

In [78]:
myFiles = myFiles[:-1]
len(myFiles)

813

In [83]:
class_id = 'shirt'
final_df=[]
for item in myFiles:
    im = Image.open('/content/gdrive/MyDrive/CEProject63-21ObjectDetection/dataset/t1/'+item.replace('txt','jpg'))
    width, height = im.size
    row=[]
    bbox_temp=[]
    with open(item, 'rt') as fd:
        first_line = fd.readline()
        splited = first_line.split();
        
        
        try:
            row.append(item)
            row.append(width)
            row.append(height)
            row.append(class_id)
            row.append(float(splited[1])*width)
            row.append(float(splited[2])*height)
            row.append(float(splited[3])*width)
            row.append(float(splited[4])*height)
            final_df.append(row)
        except:
            print("file is not in YOLO format!")
df = pd.DataFrame(final_df,columns=['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])
df.to_csv("/content/gdrive/MyDrive/CEProject63-21ObjectDetection/dataset/saved.csv",index=False)

file is not in YOLO format!


#Convert xml to csv

In [10]:
!pwd

/root/models/research


In [85]:
%cd /ce2/ssd/data/VOCdevkit/MELON/

/content/gdrive/.shortcut-targets-by-id/1hQoqbcLdIKP1y1vE2waDf28rLEbA3yqR/CEProject63-21ObjectDetection/colab_work/ssd/data/VOCdevkit/MELON


In [24]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
	'''
    for directory in ['train','testing']:
        image_path = os.path.join(os.getcwd(), 'images/{}'.format(directory).format(directory))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')
	''' 
	image_path = os.path.join(os.getcwd())
	xml_df = xml_to_csv(image_path)
	xml_df.to_csv('../train_labels.csv', index=None)

	#image_path = os.path.join(os.getcwd(), 'totoro/images/test')
	#xml_df = xml_to_csv(image_path)
	#xml_df.to_csv('totoro/data/test_labels.csv',index=None)

main()

# csv tfrecord

In [47]:
!git clone https://github.com/fllay/totoro.git #Import Totoro and Nekobus data from Github

Cloning into 'totoro'...
remote: Enumerating objects: 823, done.
remote: Total 823 (delta 0), reused 0 (delta 0), pack-reused 823
Receiving objects: 100% (823/823), 63.31 MiB | 12.63 MiB/s, done.
Resolving deltas: 100% (407/407), done.
Checking out files: 100% (822/822), done.


In [56]:
 !cp -r /root/models/totoro/tfrecord/ /ce2/ssd/data/VOCdevkit/MELON/totoro/tfrecord/

In [87]:
%cd  totoro/tfrecord/

/content/gdrive/.shortcut-targets-by-id/1hQoqbcLdIKP1y1vE2waDf28rLEbA3yqR/CEProject63-21ObjectDetection/colab_work/ssd/data/VOCdevkit/MELON/totoro/tfrecord


In [88]:
!python generate_tfrecord.py --csv_input=/ce2/ssd/data/VOCdevkit/MELON/saved.csv  --output_path=train.record --image_dir=/ce2/ssd/data/VOCdevkit/MELON/JPEGImages



W0319 11:31:48.355916 139988578473856 module_wrapper.py:139] From generate_tfrecord.py:90: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0319 11:31:48.557427 139988578473856 module_wrapper.py:139] From generate_tfrecord.py:49: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Traceback (most recent call last):
  File "generate_tfrecord.py", line 104, in <module>
    tf.app.run()
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "generate_tfrecord.py", line 95, in main
    tf_example = create_tf_example(group, path)
  File "generate_tfrecord.py", line 50, in create_tf_example
 